# Ejercicio 10: Re-ranking

**Objetivo:** Implementar y evaluar un pipeline de Recuperación de Información en dos etapas, y analizar el impacto del re-ranking en la calidad del ranking.

## Parte 1. Preparación del corpus

* Cargar el corpus (documentos/pasajes).
* Cargar las consultas (queries).
* Cargar qrels (relevancia).

In [ ]:
!pip install beir

In [ ]:
from beir import util
from beir.datasets.data_loader import GenericDataLoader
import pandas as pd

In [ ]:
DATASET_NAME = "scifact"
DATA_DIR = "../data/beir_datasets"
url = f"https://public.ukp.informatik.tu-darmstadt.de/thakur/BEIR/datasets/{DATASET_NAME}.zip"
util.download_and_unzip(url, DATA_DIR)

In [ ]:
dataset_path = DATA_DIR + "/" + DATASET_NAME
corpus, queries, qrels = GenericDataLoader(dataset_path).load(split="test")

In [ ]:
df_corpus = (
    pd.DataFrame.from_dict(corpus, orient="index")
      .reset_index()
      .rename(columns={"index": "doc_id"})
)

df_corpus

In [ ]:
df_queries = (
    pd.DataFrame.from_dict(queries, orient="index", columns=["query"])
      .reset_index()
      .rename(columns={"index": "query_id"})
)

df_queries

In [ ]:
rows = []
for qid, docs in qrels.items():
    for doc_id, rel in docs.items():
        rows.append({
            "query_id": qid,
            "doc_id": doc_id,
            "relevance": rel
        })

df_qrels = pd.DataFrame(rows)
df_qrels

In [ ]:
# Elegimos una query cualquiera que tenga varios documentos relevantes
qid = "133"

print("Query:")
print(df_queries.loc[df_queries["query_id"] == qid, "query"].values[0])

print("\nDocumentos relevantes para esta query:")
df_qrels[(df_qrels["query_id"] == qid) & (df_qrels["relevance"] > 0)]

## Parte 2. Retrieval inicial (baseline)

* Implementar retrieval inicial con BM25
* Obtener métricas: Recall@10 nDCG@10

In [ ]:
!pip install rank_bm25 nltk

In [ ]:
import pandas as pd
from rank_bm25 import BM25Okapi
import nltk
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
import re

nltk.download('punkt_tab')
nltk.download('stopwords')

stemmer = SnowballStemmer('english')
stop_words = set(stopwords.words('english'))

def clean_tokenize(text):
    # Limpieza básica: minúsculas y quitar caracteres no alfanuméricos
    text = re.sub(r'[^\w\s]', '', str(text).lower())
    # Tokenización y Stemming (solo si no es stopword)
    tokens = [stemmer.stem(w) for w in text.split() if w not in stop_words]
    return tokens

# Aplicamos al corpus
df_corpus['tokenized_text'] = df_corpus['text'].apply(clean_tokenize)
bm25 = BM25Okapi(df_corpus['tokenized_text'].tolist())

In [ ]:
results_list = []

for _, q_row in df_queries.iterrows():
    q_id = q_row['query_id']
    q_text = q_row['query']

    tokenized_query = clean_tokenize(q_text)
    scores = bm25.get_scores(tokenized_query)

    # Obtenemos los índices de los top 10
    top_n = 10
    top_indices = scores.argsort()[-top_n:][::-1]

    for rank, idx in enumerate(top_indices):
        results_list.append({
            'query_id': q_id,
            'doc_id': df_corpus.iloc[idx]['doc_id'],
            'score': scores[idx],
            'rank': rank + 1
        })

df_results = pd.DataFrame(results_list)

Ahora se procede a evaluar las metricas Recall@10 y nDCG@10

In [ ]:
!pip install ir_measures

In [ ]:
import ir_measures
from ir_measures import read_trec_run, nDCG, Recall

# 1. Adaptar formatos (asegúrate de que los IDs sean strings)
df_qrels = df_qrels.astype({'query_id': str, 'doc_id': str})
df_results = df_results.astype({'query_id': str, 'doc_id': str})

# 2. Calcular métricas
metrics = ir_measures.calc_aggregate([Recall@10, nDCG@10], df_qrels, df_results)

print("--- Resultados de la Evaluación ---")
for metric, value in metrics.items():
    print(f"{metric}: {value:.4f}")

## Parte 3. Implementación del re-ranking _cross-encoder_

* Re-rankear los top-k candidatos para cada query.
* Identificar qué documentos cambian de posición en el top 10

In [ ]:
!pip install sentence-transformers

In [ ]:
from sentence_transformers import CrossEncoder
import numpy as np

# Cargamos el modelo (puedes elegir uno específico para tu idioma)
model_re_ranker = CrossEncoder('BAAI/bge-reranker-base')

# Supongamos que re-rankeamos los top 50 de BM25 para mejorar el top 10 final
top_k_bm25 = df_results.copy()

Proceso de Reranking

In [ ]:
reranked_data = []

# Agrupamos los resultados de BM25 por query para procesarlos
for query_id, group in top_k_bm25.groupby('query_id'):
    query_text = df_queries.loc[df_queries['query_id'] == query_id, 'query'].values[0]

    # Obtenemos los textos de los documentos candidatos (vía df_corpus)
    doc_ids = group['doc_id'].tolist()
    doc_texts = df_corpus[df_corpus['doc_id'].isin(doc_ids)].set_index('doc_id').loc[doc_ids, 'text'].tolist()

    # Preparamos los pares para el Cross-Encoder
    sentence_pairs = [[query_text, doc] for doc in doc_texts]

    # Predicción de scores de relevancia
    cross_scores = model_re_ranker.predict(sentence_pairs)

    # Crear nuevos resultados
    for i in range(len(doc_ids)):
        reranked_data.append({
            'query_id': query_id,
            'doc_id': doc_ids[i],
            'bm25_rank': group.iloc[i]['rank'], # Guardamos el rango anterior
            'cross_score': cross_scores[i]
        })

df_reranked = pd.DataFrame(reranked_data)

# Ordenar por el nuevo score y asignar nuevo rank
df_reranked = df_reranked.sort_values(by=['query_id', 'cross_score'], ascending=[True, False])
df_reranked['reranked_rank'] = df_reranked.groupby('query_id').cumcount() + 1

Se identifica cambios de posición en el Top 10

In [ ]:
# Filtramos solo el nuevo Top 10
top10_changes = df_reranked[df_reranked['reranked_rank'] <= 10].copy()

# Calculamos el desplazamiento
top10_changes['shift'] = top10_changes['bm25_rank'] - top10_changes['reranked_rank']

def describe_change(row):
    if row['shift'] > 0: return f"Subió {int(row['shift'])} posiciones"
    if row['shift'] < 0: return f"Bajó {int(abs(row['shift']))} posiciones"
    return "Sin cambios"

top10_changes['status'] = top10_changes.apply(describe_change, axis=1)

# Mostrar ejemplos donde hubo movimiento
print(top10_changes[top10_changes['shift'] != 0][['query_id', 'doc_id', 'bm25_rank', 'reranked_rank', 'status']].head())

## Parte 4. Implementación del re-ranking _LTR_

* Re-rankear los top-k candidatos para cada query.
* Identificar qué documentos cambian de posición en el top 10

Preparación de Features

In [ ]:
import lightgbm as lgb
import pandas as pd
import numpy as np

# 1. Asegurémonos de rescatar el score de BM25 original
# Creamos un dataframe limpio con los scores de BM25
df_bm25_scores = df_results[['query_id', 'doc_id', 'score']].rename(columns={'score': 'score_bm25'})

# 2. Unimos con los resultados del Cross-Encoder (df_reranked)
# Usamos 'inner' para asegurarnos de tener ambos scores para cada par query-doc
df_ltr = pd.merge(
    df_reranked,
    df_bm25_scores,
    on=['query_id', 'doc_id'],
    how='inner'
)

# 3. Unimos con las etiquetas reales (qrels) para el entrenamiento
df_ltr = pd.merge(df_ltr, df_qrels, on=['query_id', 'doc_id'], how='left')
df_ltr['relevance'] = df_ltr['relevance'].fillna(0)  # Los no encontrados en qrels son 0

# 4. Ordenar por query_id es CRÍTICO para LTR
df_ltr = df_ltr.sort_values('query_id')

# 5. Ahora definimos las features con los nombres correctos
# 'score_bm25' viene de BM25 y 'cross_score' viene del Cross-Encoder
features = ['score_bm25', 'cross_score']
X = df_ltr[features]
y = df_ltr['relevance']
groups = df_ltr.groupby('query_id').size().to_list()

Entrenamiento e Identificación de Cambios

In [ ]:
# Entrenar el Ranker
ranker = lgb.LGBMRanker(
    objective="lambdarank",
    metric="ndcg",
    n_estimators=100,
    learning_rate=0.1,
    random_state=42
)

ranker.fit(X, y, group=groups)

# Generar el score final de LTR
df_ltr['ltr_score'] = ranker.predict(X)

# Ranking final
df_ltr = df_ltr.sort_values(by=['query_id', 'ltr_score'], ascending=[True, False])
df_ltr['ltr_rank'] = df_ltr.groupby('query_id').cumcount() + 1

# --- IDENTIFICAR CAMBIOS EN TOP 10 ---
# Comparamos el rank inicial (BM25) con el final (LTR)
top10_final = df_ltr[df_ltr['ltr_rank'] <= 10].copy()
top10_final['change'] = top10_final['bm25_rank'] - top10_final['ltr_rank']

print("Resumen de movimientos (Documentos que más subieron gracias a LTR):")
print(top10_final[['query_id', 'doc_id', 'bm25_rank', 'ltr_rank', 'change']].sort_values('change', ascending=False).head(10))

## Parte 5. Evaluación post re-ranking

Calcular métricas:
* nDCG@10
* MAP
* Recall@10

Preparación de los datos para la Evaluación

In [ ]:
import ir_measures
from ir_measures import nDCG, MAP, Recall

# 1. Aseguramos que los IDs sean strings para evitar errores de comparación
df_qrels_eval = df_qrels.astype({'query_id': str, 'doc_id': str})

# 2. Preparamos el DataFrame de LTR (el ranking final)
# Renombramos 'ltr_score' a 'score' porque es lo que busca la librería
df_ltr_eval = df_ltr[['query_id', 'doc_id', 'ltr_score']].copy()
df_ltr_eval.columns = ['query_id', 'doc_id', 'score']
df_ltr_eval = df_ltr_eval.astype({'query_id': str, 'doc_id': str})

# 3. Preparamos también el ranking de Cross-Encoder y BM25 para comparar
df_ce_eval = df_reranked[['query_id', 'doc_id', 'cross_score']].copy()
df_ce_eval.columns = ['query_id', 'doc_id', 'score']
df_ce_eval = df_ce_eval.astype({'query_id': str, 'doc_id': str})

df_bm25_eval = df_results[['query_id', 'doc_id', 'score']].copy()
df_bm25_eval = df_bm25_eval.astype({'query_id': str, 'doc_id': str})

Cálculo de Métricas Comparativas

In [ ]:
# Definimos las métricas deseadas
metrics_to_run = [nDCG@10, MAP, Recall@10]

# Calculamos para cada etapa
results_bm25 = ir_measures.calc_aggregate(metrics_to_run, df_qrels_eval, df_bm25_eval)
results_ce   = ir_measures.calc_aggregate(metrics_to_run, df_qrels_eval, df_ce_eval)
results_ltr  = ir_measures.calc_aggregate(metrics_to_run, df_qrels_eval, df_ltr_eval)

# Formateamos los resultados en una tabla comparativa
eval_df = pd.DataFrame([results_bm25, results_ce, results_ltr],
                       index=['BM25 (Base)', 'Cross-Encoder (Re-rank)', 'LTR (Final)'])

print("--- REPORTE FINAL DE MÉTRICAS ---")
print(eval_df.round(4))